In [2]:
import sys
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing.image import load_img, img_to_array
import numpy as np

sys.path.append('pyttr')
from pyttr import ttrtypes as ttr, utils as ttru

Using TensorFlow backend.


## ResNet50 classification

In [3]:
model = ResNet50()

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [4]:
def classify(path):
    img = load_img(path, target_size=(224,224))
    input = preprocess_input(np.array([img_to_array(img)]))
    prediction = decode_predictions(model.predict(input))[0]
    return prediction[0][1].replace('_', ' ')
classify('res/canal.jpg')

'palace'

In [5]:
M = ttr.Possibility('M1')
Ind = ttr.BType('Ind')
Ind.in_poss(M)

def see(path):
    pred = ttr.Pred(classify(path), [Ind])
    name = path.split('/')[-1].split('.')[0]
    Ind.judge(name)
    ttr.PType(pred, [name]).in_poss(M).judge(path)

In [6]:
see('res/volcano.jpg')
see('res/volcano2.jpg')
see('res/canal.jpg')
see('res/dogcar.jpg')
#see('/home/arildm/Pictures/star.png')
#see('/home/arildm/Pictures/skocaj.png')
print(ttru.show(M))


M1:
_____________________________________________
Ind: [volcano, volcano2, canal, dogcar]
palace(canal): [res/canal.jpg]
volcano(volcano2): [res/volcano2.jpg]
volcano(volcano): [res/volcano.jpg]
Norfolk terrier(dogcar): [res/dogcar.jpg]
_____________________________________________



## Classification as a witness condition

In [20]:
words = ['volcano', 'palace', 'dog', 'canal']
ptypes = []
for word in words:
    ptype = ttr.PType(ttr.Pred(word, [Ind]), ['x'])
    ptype.learn_witness_condition(lambda path, word=word: classify(path) == word)
    ptypes.append(ptype)
for path in ['res/volcano2.jpg', 'res/canal.jpg', 'res/dogcar.jpg']:
    for ptype in ptypes:
        if ptype.query(path):
            print(path + ' is a ' + ptype.show())

res/volcano2.jpg is a volcano(x)
res/canal.jpg is a palace(x)


### Questionable "object detector" function

Still with ResNet50, meaning only one class for the image, no object detection.

In [21]:
imageT = ttr.BType('Image')

def odf(path):
    det = set()
    for ptype in ptypes:
        if ptype.query(path):
            det.add(ttr.RecType({
                'reg': str(load_img(path)),
                'pfun': ttr.Fun('x', imageT, ptype),
            }))
    return det
            
print([t.show() for t in odf('res/volcano2.jpg')])

['{pfun : lambda x:Image . volcano(x), reg : <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1200x900 at 0x7FCB86B85080>}']


## YOLO

Enter object detection.

Requires OpenCV and [Darkflow](https://github.com/thtrieu/darkflow). yolo.weights is from [Yolo](https://pjreddie.com/darknet/yolo/).

In [45]:
from darkflow.net.build import TFNet
import cv2

options = {"model": "yolo/yolo.cfg", "load": "yolo/yolo.weights", "threshold": 0.1}

tfnet = TFNet(options)

/opt/darkflow/darkflow/dark/darknet.py:54: UserWarning: ./cfg/yolo.cfg not found, use yolo/yolo.cfg instead
  cfg_path, FLAGS.model))


Parsing yolo/yolo.cfg
Loading yolo/yolo.weights ...
Successfully identified 203934260 bytes
Finished in 0.014590978622436523s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Load  |  Yep!  | conv 3x3p1_1  +b

In [42]:
def object_detector(cv_image):
    s = set()
    for o in tfnet.return_predict(imgcv):
        if o['confidence'] > 0.1:
            ptype = ttr.PType(ttr.Pred(o['label'], [Ind]), ['x'])
            pfun = ttr.Fun('x', imageT, ptype)
            s.add(ttr.RecType({'reg': (o['topleft'], o['bottomright']), 'pfun': pfun}))
    return s

In [43]:
for t in object_detector(cv2.imread('res/dogcar.jpg')):
    print(t.show())

{pfun : lambda x:Image . car(x), reg : ({x = 170, y = 89}, {x = 1019, y = 1006})}
{pfun : lambda x:Image . chair(x), reg : ({x = 191, y = 308}, {x = 905, y = 1079})}
{pfun : lambda x:Image . person(x), reg : ({x = 356, y = 236}, {x = 1065, y = 1072})}
{pfun : lambda x:Image . dog(x), reg : ({x = 241, y = 320}, {x = 968, y = 1079})}
{pfun : lambda x:Image . dog(x), reg : ({x = 706, y = 574}, {x = 1079, y = 1067})}
{pfun : lambda x:Image . person(x), reg : ({x = 3, y = 697}, {x = 386, y = 1079})}


Now what about the witness condition??